In [108]:
import pyspark

In [109]:
from pyspark.sql import SparkSession

In [110]:
spark = SparkSession.builder.appName('week5GraphTask2').enableHiveSupport().master('yarn').getOrCreate()

In [111]:
spark

In [112]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number,col,sum,count

In [113]:
data = spark.read.parquet("/data/sample264")

In [115]:
# data.show()

In [116]:
datagrp = data.groupBy(col('artistId'),col('trackId')).agg(count(col('userId')).alias('ct'))

In [117]:
window = Window.partitionBy(col('artistId')).orderBy(col('ct').desc())

In [118]:
# datagrp.filter(col('userId') == 13065).show()

In [119]:
datalmt = datagrp.withColumn('row_number',row_number().over(window))\
    .filter(col('row_number')<= 1000).drop(col('row_number'))

In [120]:
# datalmt.show()

In [121]:
datalmtgrp = datalmt.groupBy(col('artistId')).agg(sum(col('ct')).alias('sum_ct'))

In [122]:
data_norm  = datagrp.join(datalmtgrp,'artistId','inner').withColumn('norm',col('ct')/col('sum_ct'))

In [125]:
res = data_norm.orderBy(col('norm').desc(),col('artistId'),col('trackId')).limit(40)

In [126]:
res_final = res.select(col('artistId'),col('trackId'))

In [127]:
for t1,t2 in res_final.collect():
    print('{}\t{}'.format(t1,t2))

967993	869415
967998	947428
968004	927380
968017	859321
968022	852786
968034	807671
968038	964150
968042	835935
968043	913568
968046	935077
968047	806127
968065	907906
968073	964586
968086	813446
968092	837129
968118	914441
968125	821410
968140	953008
968148	877445
968161	809793
968163	803065
968168	876119
968189	858639
968221	896937
968224	892880
968232	825536
968237	932845
968238	939177
968241	879045
968242	911250
968248	953554
968255	808494
968259	880230
968265	950148
968266	824437
968269	913243
968272	816049
968278	946743
968285	847460
968286	940006
